In [12]:
import pandas as pd
from sklearn.metrics.pairwise import pairwise_distances

# Load the ratings data
ratings = pd.read_csv('ratings.csv')

# Load the movies data
movies = pd.read_csv('movies.csv')

# Create a user-item matrix
user_item_matrix = ratings.pivot_table(index='userId', columns='movieId', values='rating')

# Calculate the cosine similarity between users
user_similarity = 1 - pairwise_distances(user_item_matrix.fillna(0), metric='cosine')

# Choose the top K most similar users to user A
k = 10
similar_users = user_similarity.argsort()[:,::-1][:,:k+1][:,1:]
valid_users = user_item_matrix.columns.isin(user_item_matrix.columns[similar_users.flatten()])
movies_not_seen_by_a = user_item_matrix.loc[1, valid_users].isnull()

# Calculate the recommendation scores for all items
movie_scores = user_item_matrix.mean().where(movies_not_seen_by_a, other=0)

# Join the movie titles to the recommendation scores
recommended_movies = pd.merge(movies, pd.DataFrame(movie_scores), left_on='movieId', right_index=True)
recommended_movies = recommended_movies.sort_values(by=0, ascending=False).head(10)
recommended_movies = recommended_movies[['title', 0]].rename(columns={0: 'score'})

# Print the recommended movies
print("Recommended movies:")
print(recommended_movies)

Recommended movies:
                                                 title  score
536                             Denise Calls Up (1995)   5.00
87                Heidi Fleiss: Hollywood Madam (1995)   5.00
531          Thin Line Between Love and Hate, A (1996)   5.00
48                                     Lamerica (1994)   5.00
433                        What Happened Was... (1994)   5.00
405                             Live Nude Girls (1995)   5.00
432  In the Realm of the Senses (Ai no corrida) (1976)   5.00
121                   Awfully Big Adventure, An (1995)   5.00
557             World of Apu, The (Apur Sansar) (1959)   4.75
441                                     Orlando (1992)   4.50
